# Part 5: Building the Multimodal Model

We incorporate our learnings from part 4 into building a Multimodal Model with different processing pathways for the various modalities, and then merge their processed features for the final classification. 

In [9]:
# Import necessary libraries

import os
import sys
import copy
import pickle
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import keras.api._v2.keras as keras
import keras.backend as K
from keras.optimizers.legacy import Adam
from keras.optimizers import SGD
from keras import callbacks
from keras.models import Model
from keras.layers import Masking, Layer, Dense, Input, Activation, Lambda, BatchNormalization, Conv1D, SpatialDropout1D, Add, GlobalAveragePooling1D, LSTM, Dense, concatenate, TimeDistributed, Bidirectional, Dropout, Embedding, Attention, MultiHeadAttention, LayerNormalization, Flatten, Concatenate
from tensorflow.keras.backend import sigmoid
from sklearn.model_selection import KFold
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support, accuracy_score
import tensorflow as tf
from data_utils import load_data
from model_utils import *
import tensorflow as tf
from collections import Counter

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Load data
train_data, train_label, train_mask, test_data, test_label, test_mask, train_text, train_audio, train_visual, test_text, test_audio, test_visual = load_data()

Max length of sequences:  110

Number of training samples: 120
Number of testing samples: 31

Train text shape: 120 samples, 110 timesteps, 100 features
Train audio shape: 120 samples, 110 timesteps, 100 features
Train visual shape: 120 samples, 110 timesteps, 512 features

Test text shape: 31 samples, 110 timesteps, 100 features
Test audio shape: 31 samples, 110 timesteps, 100 features
Test visual shape: 31 samples, 110 timesteps, 512 features



In [22]:
train_mask = tf.convert_to_tensor(train_mask, dtype=tf.float32)
test_mask = tf.convert_to_tensor(test_mask, dtype=tf.float32)

## Transformer Block and Cross-Modal Attention

In [3]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.mha = MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=False):
        attn_output = self.mha(inputs, inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

class CrossModalAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.multi_head_attn = MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
    
    def call(self, query, key_value):
        attn_output = self.multi_head_attn(query, key_value, key_value)
        return attn_output
    
def masked_categorical_crossentropy(y_true, y_pred, mask):
    # Calculate the categorical crossentropy loss
    loss = tf.keras.losses.categorical_crossentropy(y_true, y_pred)
    
    # Mask the loss using the pre-computed mask
    loss *= mask
    
    # Average the loss, ignoring the padded timesteps
    return tf.reduce_sum(loss) / tf.reduce_sum(mask)

## Multimodal model with cross-modal attention

This model processes the individual modalities through different pathways. Then, it uses cross-modal attention layers to study the dependencies between text and video, and text and audio, prioritising text as the most important modality.

In [17]:
class MultimodalModel(BaseModel):
    def __init__(self, num_classes, d_model, num_heads):
        super(MultimodalModel, self).__init__()
        
        # Text feature processing using bi-LSTM
        self.text_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.text_bilstm_1 = Bidirectional(LSTM(64, return_sequences=True))
        self.text_bilstm_2 = Bidirectional(LSTM(64, return_sequences=True))
        self.text_dense = Dense(d_model, activation='relu')

        # Audio feature processing using 1D CNN with padding
        self.audio_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.audio_cnn_1 = Conv1D(filters=128, kernel_size=8, activation='relu', padding='same')
        self.audio_cnn_2 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')
        self.audio_dense = Dense(d_model, activation='relu')

        # Video feature processing using CNN and Multi-Head Attention
        self.video_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.video_attn = MultiHeadAttention(num_heads=8, key_dim=64)
        self.video_cnn_1 = Conv1D(128, kernel_size=8, activation='relu', padding='same')
        self.video_cnn_2 = Conv1D(128, kernel_size=3, activation='relu', padding='same')
        self.video_dense = Dense(d_model, activation='relu')

        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Final processing and classification layers
        self.concat = Concatenate()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training):
        audio_features, text_features, video_features = inputs
        
        # Process video features and reduce dimensionality
        video_features = self.video_mask(video_features)
        video_features = self.video_attn(video_features, video_features)
        video_features = self.video_cnn_1(video_features)
        video_features = self.video_cnn_2(video_features)
        video_features = self.video_dense(video_features)

        # Process text features
        text_features = self.text_mask(text_features)
        text_features = self.text_bilstm_1(text_features)
        text_features = self.text_bilstm_2(text_features)
        text_features = self.text_dense(text_features)

        # Process audio features
        audio_features = self.audio_mask(audio_features)
        audio_features = self.audio_cnn_1(audio_features)
        audio_features = self.audio_cnn_2(audio_features)
        audio_features = self.audio_dense(audio_features)
        
        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(text_features, audio_features)
        text_video_attn = self.text_video_attention(text_features, video_features)
        
        # Combine features from both attention mechanisms
        combined_features = self.concat([text_features, text_audio_attn, text_video_attn])
        outputs = self.classifier(combined_features)

        # Return the final prediction
        return outputs


In [23]:
# Instantiate the model
multimodal_model = MultimodalModel(num_classes=5, d_model=100, num_heads=8)

# Compile the model
multimodal_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodal_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodal_model.evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodal_model.print_metrics()

Epoch 1/10


ValueError: in user code:

    File "/Users/venessa/Desktop/5342project/venv/lib/python3.8/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/var/folders/6l/xwrqvy7n1814gqhygtglkblw0000gn/T/ipykernel_93552/792049276.py", line 2, in None  *
        lambda y_true, y_pred: masked_categorical_crossentropy(y_true, y_pred, train_mask)
    File "/Users/venessa/Desktop/5342project/model_utils.py", line 15, in masked_categorical_crossentropy  *
        loss *= mask

    ValueError: Dimensions must be equal, but are 32 and 120 for '{{node lambda/mul}} = Mul[T=DT_FLOAT](lambda/softmax_cross_entropy_with_logits/Reshape_2, lambda/mul/y)' with input shapes: [32,110], [120,110].


In [21]:
type(train_mask)

numpy.ndarray

## Multimodal Model with Self-Attention

This model first adds a layer of self-attention for each modality, before processing the individual modalities through different pathways. Then, similar to above, it uses cross-modal attention layers to study the dependencies between text and video, and text and audio, prioritising text as the most important modality.

It was found that the added layer of attention worsened performance.

In [ ]:
class MultimodalAttentionModel(BaseModel):
    def __init__(self, num_classes, d_model=128, num_heads=4):
        super(MultimodalAttentionModel, self).__init__()
        
        # Text Pathway - Bi-LSTM with Attention
        self.text_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.text_attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.text_bi_lstm1 = Bidirectional(LSTM(64, return_sequences=True))
        self.text_bi_lstm2 = Bidirectional(LSTM(64, return_sequences=True))
        self.text_dropout = Dropout(0.5)
        self.text_dense = Dense(100, activation='relu')
        
        # Audio Pathway - CNN with Attention
        self.audio_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.audio_attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.audio_conv1 = Conv1D(100, kernel_size=10, activation='relu', padding='same')
        self.audio_conv2 = Conv1D(100, kernel_size=9, activation='relu', padding='same')
        self.audio_conv3 = Conv1D(100, kernel_size=8, activation='relu', padding='same')
        self.audio_dense = Dense(100, activation='relu')
        
        # Video Pathway - Similar to Audio
        self.video_mask = Masking(mask_value=0.0, input_shape=(None, 110))
        self.video_attention = MultiHeadAttention(num_heads=num_heads, key_dim=d_model // num_heads)
        self.video_conv1 = Conv1D(100, kernel_size=10, activation='relu', padding='same')
        self.video_conv2 = Conv1D(100, kernel_size=9, activation='relu', padding='same')
        self.video_conv3 = Conv1D(100, kernel_size=8, activation='relu', padding='same')
        self.video_dense = Dense(100, activation='relu')
        
        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Final processing and classification layers
        self.concat = Concatenate()
        self.global_pool = GlobalAveragePooling1D()
        self.classifier = Dense(num_classes, activation='softmax')
        
    def call(self, inputs, training=False):
        text_inputs, audio_inputs, video_inputs = inputs
        
        # Text Pathway
        x_text = self.text_mask(text_inputs)
        x_text = self.text_attention(x_text, x_text)
        x_text = self.text_bi_lstm1(x_text)
        x_text = self.text_bi_lstm2(x_text)
        x_text = self.text_dropout(x_text)
        x_text = self.text_dense(x_text)
        
        # Audio Pathway
        x_audio = self.audio_mask(audio_inputs)
        x_audio = self.audio_attention(x_audio, x_audio)
        x_audio = self.audio_conv1(x_audio)
        x_audio = self.audio_conv2(x_audio)
        x_audio = self.audio_conv3(x_audio)
        x_audio = self.audio_dense(x_audio)
        
        # Video Pathway
        x_video = self.video_mask(video_inputs)
        x_video = self.video_attention(x_video, x_video)
        x_video = self.video_conv1(x_video)
        x_video = self.video_conv2(x_video)
        x_video = self.video_conv3(x_video)
        x_video = self.video_dense(x_video)

        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(x_text, x_audio)
        text_video_attn = self.text_video_attention(x_text, x_video)
        
        # Combine features from both attention mechanisms
        combined_features = self.concat([x_text, text_audio_attn, text_video_attn])
        outputs = self.classifier(combined_features)

        # Return the final prediction
        return outputs
    

In [ ]:
# Instantiate the model
multimodal_attention_model = MultimodalAttentionModel(num_classes=5)

# Compile the model
multimodal_attention_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodal_attention_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodal_attention_model.evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodal_attention_model.print_metrics()

## Multimodal Model with simplified preprocessing
This model applies less processing to the individual modalities. Video features are processed through a transformer block and their dimensionality is reduced. Again, cross-modal attention is used.

In [ ]:
class MultimodalTransformerModel(Model):
    def __init__(self, d_model, num_heads, dff, rate=0.1, num_classes=10):
        super(MultimodalTransformerModel, self).__init__()

        # Video feature processing using Transformer block
        self.video_transformer = TransformerBlock(512, num_heads, dff, rate)
        self.video_dense = Dense(64)
        
        # Text feature processing using bi-LSTM
        self.text_lstm = Bidirectional(LSTM(64, return_sequences=True))
        self.text_dense = Dense(64, activation='relu')

        # Audio feature processing using 1D CNN with padding
        self.audio_cnn = Conv1D(filters=128, kernel_size=5, activation='relu', padding='same')
        self.audio_dense = Dense(64, activation='relu')

        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Final processing and classification layers
        self.concat = Concatenate()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training):
        audio_features, text_features, video_features = inputs
        
        # Process video features and reduce dimensionality
        video_features = self.video_transformer(video_features, training=training)
        video_features = self.video_dense(video_features)

        # Process text features
        text_features = self.text_lstm(text_features)
        text_features = self.text_dense(text_features)

        # Process audio features
        audio_features = self.audio_cnn(audio_features)
        audio_features = self.audio_dense(audio_features)
        
        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(text_features, audio_features)
        text_video_attn = self.text_video_attention(text_features, video_features)
        
        # Combine features from both attention mechanisms
        combined_features = self.concat([text_features, text_audio_attn, text_video_attn])
        outputs = self.classifier(combined_features)

        # Return the final prediction
        return outputs
    
    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        self.test_metrics =  {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'cm': confusion_matrix(y_true, y_pred)
        }

    def print_metrics(self):
        print("Test Metrics:")
        print(f"Accuracy: {self.test_metrics['accuracy']:.4f}")
        # print(f"Precision: {metrics['precision']:.4f}")
        # print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {self.test_metrics['f1_score']:.4f}")

        class_labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
        print("Confusion Matrix:")
        plt.figure(figsize=(8,6))
        sns.heatmap(self.test_metrics['cm'], annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()

In [ ]:
# Instantiate the model
multimodal_transformer_model = MultimodalTransformerModel(d_model=100, num_heads=4, dff=2048, rate=0.1, num_classes=6)

# Compile the model
multimodal_transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodal_transformer_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodal_transformer_model.evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodal_transformer_model.print_metrics()

## Multimodal Transformer Model

In [ ]:
class MultimodalTransformerModel(Model):
    def __init__(self, d_model, num_heads, dff, rate=0.1, num_classes=10):
        super(MultimodalTransformerModel, self).__init__()
        # Transformer block for processing video features
        self.video_transformer = TransformerBlock(d_model=512, num_heads=8, dff=dff, rate=rate)
        self.video_dense = Dense(d_model)
        
        # Text feature processing using bi-LSTM
        self.text_transformer = TransformerBlock(d_model=100, num_heads=4, dff=dff, rate=rate)
        self.text_dense = Dense(d_model)

        # Audio feature processing using 1D CNN with padding
        self.audio_transformer = TransformerBlock(d_model=100, num_heads=4, dff=dff, rate=rate)
        self.audio_dense = Dense(d_model)

        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Final processing and classification layers
        self.concat = Concatenate()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training):
        audio_features, text_features, video_features = inputs
        
        # Process video features and reduce dimensionality
        video_features = self.video_transformer(video_features, training=training)
        video_features = self.video_dense(video_features)

        # Process text features
        text_features = self.text_transformer(text_features, training=training)
        text_features = self.text_dense(text_features)

        # Process audio features
        audio_features = self.audio_transformer(audio_features, training=training)
        audio_features = self.audio_dense(audio_features)
        
        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(text_features, audio_features)
        text_video_attn = self.text_video_attention(text_features, video_features)
        
        # Combine features from both attention mechanisms
        combined_features = self.concat([text_features, text_audio_attn, text_video_attn])
        outputs = self.classifier(combined_features)

        # Return the final prediction
        return outputs
    
    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        self.test_metrics =  {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'cm': confusion_matrix(y_true, y_pred)
        }

    def print_metrics(self):
        print("Test Metrics:")
        print(f"Accuracy: {self.test_metrics['accuracy']:.4f}")
        # print(f"Precision: {metrics['precision']:.4f}")
        # print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {self.test_metrics['f1_score']:.4f}")

        class_labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
        print("Confusion Matrix:")
        plt.figure(figsize=(8,6))
        sns.heatmap(self.test_metrics['cm'], annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()

In [ ]:
# Instantiate the model
multimodal_transformer_model = MultimodalTransformerModel(num_classes=6, d_model=100, num_heads=4, dff=2048, rate=0.1)

# Compile the model
multimodal_transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodal_transformer_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodal_transformer_model.evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodal_transformer_model.print_metrics()

## TIMNet Multimodal Model

The models below incorporate TIMNet as part of their architecture. Different architectures are explored to find the optimal one.

In [ ]:
class TemporalAwareBlock(Layer):
    def __init__(self, nb_filters, kernel_size, dilation_rate, activation, dropout_rate, name=''):
        super(TemporalAwareBlock, self).__init__(name=name)
        self.nb_filters = nb_filters
        self.kernel_size = kernel_size
        
        # Primary layers used in each temporal block
        self.conv1 = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')
        self.norm1 = BatchNormalization()
        self.act1 = Activation(activation)
        self.dropout1 = SpatialDropout1D(dropout_rate)

        self.conv2 = Conv1D(filters=nb_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, padding='causal')
        self.norm2 = BatchNormalization()
        self.act2 = Activation(activation)
        self.dropout2 = SpatialDropout1D(dropout_rate)
        
        # Additional Conv1D layer for matching dimensions if needed
        self.dim_match_conv = Conv1D(filters=nb_filters, kernel_size=1, padding='same')

    def call(self, inputs):
        original_x = inputs

        x = self.conv1(inputs)
        x = self.norm1(x)
        x = self.act1(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.norm2(x)
        x = self.act2(x)
        output_2_1 = self.dropout2(x)

        # Check and apply dimension matching if necessary
        if original_x.shape[-1] != output_2_1.shape[-1]:
            original_x = self.dim_match_conv(original_x)

        # Applying sigmoid and element-wise multiplication
        output_2_1 = Lambda(sigmoid)(output_2_1)
        F_x = Lambda(lambda x: tf.multiply(x[0], x[1]))([original_x, output_2_1])
        return F_x


class TIMNET(Model):
    def __init__(self, nb_filters=64, kernel_size=2, nb_stacks=1, dilations=None, activation="relu", dropout_rate=0.1, return_sequences=True, name='TIMNET'):
        super(TIMNET, self).__init__(name=name)
        self.nb_stacks = nb_stacks
        self.dilations = dilations or [1, 2, 4, 8]
        self.forward_blocks = []
        self.backward_blocks = []

        for _ in range(nb_stacks):
            for dilation in self.dilations:
                self.forward_blocks.append(TemporalAwareBlock(nb_filters, kernel_size, dilation, activation, dropout_rate))
                self.backward_blocks.append(TemporalAwareBlock(nb_filters, kernel_size, dilation, activation, dropout_rate))

    def call(self, inputs):
        forward = inputs
        backward = tf.reverse(inputs, axis=[1])  # Assuming the time dimension is axis 1

        final_skip_connections = []

        for forward_block, backward_block in zip(self.forward_blocks, self.backward_blocks):
            skip_out_forward = forward_block(forward)
            skip_out_backward = backward_block(backward)
            # Combine skip outputs from forward and backward blocks at the same level
            temp_skip = Add()([skip_out_forward, skip_out_backward])
            final_skip_connections.append(temp_skip)

        # Combine all skip connections
        if final_skip_connections:
            output = final_skip_connections[0]
            for skip_connection in final_skip_connections[1:]:
                output = concatenate([output, skip_connection], axis=-1)

        return output


1. Using TIMNet on already combined features, similar to initial run of TIMNet. The results are worse than the original TIMNet, but that could be due to less epochs, or other factors.

In [ ]:
class MultiModalNet(Model):
    def __init__(self, num_classes=6, d_model=128, num_heads=4):
        super(MultiModalNet, self).__init__()
        # Define pathways for text, audio, and video
        self.timnet = TIMNET(nb_filters=64, kernel_size=3, nb_stacks=2, dilations=[1, 2, 4, 8], activation="relu", dropout_rate=0.2)
        self.concat = Concatenate()
        self.classifier = TimeDistributed(Dense(num_classes, activation='softmax'))


    def call(self, inputs, training=False):
        
        timnet = self.timnet(inputs)

        outputs = self.classifier(timnet)

        return outputs

    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        self.test_metrics =  {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'cm': confusion_matrix(y_true, y_pred)
        }
    
    def print_metrics(self):
        print("Test Metrics:")
        print(f"Accuracy: {self.test_metrics['accuracy']:.4f}")
        # print(f"Precision: {metrics['precision']:.4f}")
        # print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {self.test_metrics['f1_score']:.4f}")

        class_labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
        print("Confusion Matrix:")
        plt.figure(figsize=(8,6))
        sns.heatmap(self.test_metrics['cm'], annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()


In [ ]:
# Instantiate the model
multimodalnet_model = MultiModalNet(num_classes=6)

# Compile the model
multimodalnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodalnet_model.fit(train_data, train_label, epochs=50, batch_size=64)

# Evaluate the model
multimodalnet_model.evaluate(test_data, test_label)

# Print the metrics
multimodalnet_model.print_metrics()

2. Using TIMNet to preprocess individual modalities before applying cross-modal attention. Very slight improvement from the previous one.

In [ ]:
class MultiModalNet(Model):
    def __init__(self, num_classes=6, d_model=128, num_heads=4):
        super(MultiModalNet, self).__init__()
        # Define pathways for text, audio, and video
        self.text_timnet = TIMNET(nb_filters=64, kernel_size=3, nb_stacks=2, dilations=[1, 2, 4], activation="relu", dropout_rate=0.2)
        self.audio_timnet = TIMNET(nb_filters=64, kernel_size=3, nb_stacks=2, dilations=[1, 2, 4], activation="relu", dropout_rate=0.2)
        self.video_timnet = TIMNET(nb_filters=64, kernel_size=3, nb_stacks=2, dilations=[1, 2, 4], activation="relu", dropout_rate=0.2)
        
        # Cross modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        
        self.concat = Concatenate()
        self.classifier = TimeDistributed(Dense(num_classes, activation='softmax'))


    def call(self, inputs, training=False):
        text_inputs, audio_inputs, video_inputs = inputs

        x_text = text_inputs
        x_audio = audio_inputs
        x_video = video_inputs

        # TIMNET processing
        x_text = self.text_timnet(x_text)
        x_audio = self.audio_timnet(x_audio)
        x_video = self.video_timnet(x_video)

        # Apply cross-modal attention between audio-video and text-video
        text_audio_attn = self.text_audio_attention(x_text, x_audio)
        text_video_attn = self.text_video_attention(x_text, x_video)

        # Combine and classify
        combined_features = self.concat([x_text, text_audio_attn, text_video_attn])
        outputs = self.classifier(combined_features)

        return outputs

    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        self.test_metrics =  {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'cm': confusion_matrix(y_true, y_pred)
        }
    
    def print_metrics(self):
        print("Test Metrics:")
        print(f"Accuracy: {self.test_metrics['accuracy']:.4f}")
        # print(f"Precision: {metrics['precision']:.4f}")
        # print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {self.test_metrics['f1_score']:.4f}")

        class_labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
        print("Confusion Matrix:")
        plt.figure(figsize=(8,6))
        sns.heatmap(self.test_metrics['cm'], annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()


In [ ]:
# Instantiate the model
multimodalnet_model = MultiModalNet(num_classes=6)

# Compile the model
multimodalnet_model .compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodalnet_model .fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodalnet_model .evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodalnet_model.print_metrics()

3. Processing each modality with individual pathways before cross-modal attention, and finally TIMNet on concatenated features

In [ ]:
class MultiModalNet(Model):
    def __init__(self, num_classes=6, d_model=128, num_heads=4, nb_filters=64, kernel_size=3, nb_stacks=1, dilations=[1, 2, 4, 8]):
        super(MultiModalNet, self).__init__()
        # Initialize processing layers for each modality
        self.text_lstm = Bidirectional(LSTM(50, return_sequences=True))
        self.audio_cnn = Conv1D(filters=100, kernel_size=3, activation='relu', padding='same')
        self.video_cnn = Conv1D(filters=100, kernel_size=3, activation='relu', padding='same')

        # Cross-modal attention layers
        self.text_audio_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)
        self.text_video_attention = CrossModalAttentionLayer(d_model=d_model, num_heads=num_heads)

        # Concatenation layer
        self.concat = Concatenate()

        # TIMNET applied after cross-modal attention
        self.timnet = TIMNET(nb_filters=nb_filters, kernel_size=kernel_size, nb_stacks=nb_stacks, dilations=dilations, activation="relu", dropout_rate=0.1)

        # Classifier
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=False):
        text_inputs, audio_inputs, video_inputs = inputs

        # Process each modality
        x_text = self.text_lstm(text_inputs)
        x_audio = self.audio_cnn(audio_inputs)
        x_video = self.video_cnn(video_inputs)

        # Apply cross-modal attention
        text_audio_attn = self.text_audio_attention(x_text, x_audio)
        text_video_attn = self.text_video_attention(x_text, x_video)

        # Combine features from attention mechanisms
        combined_features = self.concat([x_text, text_audio_attn, text_video_attn])

        # Pass combined features through TIMNET
        processed_features = self.timnet(combined_features)

        # Pool and classify
        outputs = self.classifier(processed_features)
        
        return outputs


    def evaluate(self, x_test, y_test):
        predictions = self.predict(x_test)
        y_pred = np.argmax(predictions, axis=-1).flatten()
        y_true = np.argmax(y_test, axis=-1).flatten()

        accuracy = accuracy_score(y_true, y_pred)
        precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='macro')

        self.test_metrics =  {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'cm': confusion_matrix(y_true, y_pred)
        }
    
    def print_metrics(self):
        print("Test Metrics:")
        print(f"Accuracy: {self.test_metrics['accuracy']:.4f}")
        # print(f"Precision: {metrics['precision']:.4f}")
        # print(f"Recall: {metrics['recall']:.4f}")
        print(f"F1 Score: {self.test_metrics['f1_score']:.4f}")

        class_labels = ['hap', 'sad', 'neu', 'ang', 'exc', 'fru']
        print("Confusion Matrix:")
        plt.figure(figsize=(8,6))
        sns.heatmap(self.test_metrics['cm'], annot=True, fmt='g', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
        plt.xlabel('Predicted labels')
        plt.ylabel('True labels')
        plt.title('Confusion Matrix')
        plt.show()


In [ ]:
# Instantiate the model
multimodalnet_model = MultiModalNet(num_classes=6)

# Compile the model
multimodalnet_model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
multimodalnet_model.fit([train_audio, train_text, train_visual], train_label, epochs=10, batch_size=32)

# Evaluate the model
multimodalnet_model.evaluate([test_audio, test_text, test_visual], test_label)

# Print the metrics
multimodalnet_model.print_metrics()